In [527]:
# # os.remove("data.db")
# con = sqlite3.connect("data.db")
# con.row_factory = sqlite3.Row
# cur = con.cursor()
# cur.execute("CREATE TABLE Persons (name text, age int)")
# cur.execute("INSERT INTO Persons VALUES ('Peter', 32)")
# cur.execute("SELECT * FROM Persons")
# [{k:r[k] for k in r.keys()}for r in cur.execute('SELECT * from sqlite_master').fetchall()]

In [528]:
import base64 ,os, time, json, pickle, subprocess, sqlite3
from urllib.parse import quote
import pandas as pd

# Helper Functions

In [529]:
def hex2str(hex):
    """
    Decodes a hex string into a regular string
    """
    return bytes.fromhex(hex[2:]).decode("utf-8")

def str2hex(str):
    """
    Encodes a string as a hex string
    """
    return "0x" + str.encode("utf-8").hex()

In [530]:
def input_box(address, private_key, payload):
    input_box_abi = json.loads(open("/Users/shahnawazakhtar/Desktop/liquid-dao/deployments/localhost/InputBox.json").read())["abi"]
    input_box_address = w3.toChecksumAddress("0x5a723220579c0dcb8c9253e6b4c62e572e379945")
    dapp_address = w3.toChecksumAddress("0x142105FC8dA71191b3a13C738Ba0cF4BC33325e2")    # From dapp.json file     

    contract = w3.eth.contract(address=input_box_address, abi=input_box_abi)
    function = contract.functions['addInput'](dapp_address, str2hex(json.dumps(payload)))

    # Create a transaction
    transaction = function.buildTransaction({
        'chainId': 31337,  # Mainnet, change it to the desired network ID (e.g., 3 for Ropsten)
        'gas': 200000,  # Adjust the gas limit as required
        'gasPrice': w3.toWei('50', 'gwei'),  # Adjust the gas price as required
        'nonce': w3.eth.getTransactionCount(address),
    })
    
    # Sign the transaction with your private key, sent it and wait for receipt
    signed_transaction = w3.eth.account.signTransaction(transaction, private_key=private_key)
    tx_hash = w3.eth.sendRawTransaction(signed_transaction.rawTransaction)
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    print(tx_hash)

# Calls that can be made at genesis

In [531]:
default_address = "0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266"
default_pkey = "0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80"

def generate_call(calltype, action, metadata, address = default_address, private_key=default_pkey):
    data = {
        "action": action,
        "metadata": metadata}
    input_box(address,private_key,data)

def create_ammendment_proposal(name, logic, description, address = default_address, private_key=default_pkey):
    data = {
        "action": "create_ammendment_proposal",
        "metadata": {
            "name": name,
            "logic": str2hex(logic),
            "description": description
        }}
    input_box(address,private_key,data)


def delete_ammendment_proposal(name, address = default_address, private_key=default_pkey):
    data = {
        "action": "delete_ammendment_proposal",
        "metadata": {
            "name": name
        }}
    input_box(address,private_key,data)

def ammendment_concensus(name, address = default_address, private_key=default_pkey):
    data = {
        "action": "ammendment_concensus",
        "metadata": {
            "name": name
        }}
    input_box(address,private_key,data)

    
def table_info(name):
    data = {
        "action": "table_info",
        "metadata": {
            "name": name
        }}
    response = requests.get("http://localhost:5005/inspect/"+quote(json.dumps(data)))
    return pd.DataFrame(json.loads(hex2str(response.json()['reports'][0]['payload'])))
    
def notices():
    os.chdir("/Users/shahnawazakhtar/Desktop/liquid-dao/frontend-console")
    os.system("yarn start notice list")

# Bootstrapping the DAO

#### At Genesis

In [540]:
table_info("RULES")

,name,logic,description
0,create_ammendment_proposal,0x0a646566206372656174655f616d6d656e646d656e74...,Create ammendment proposal
1,delete_ammendment_proposal,0x0a6465662064656c6574655f616d6d656e646d656e74...,Delete ammendment proposal
2,ammendment_concensus,0x0a64656620616d6d656e646d656e745f636f6e63656e...,Rules for ammendment_concensus in the DAO at g...
3,table_info,0x0a646566207461626c655f696e666f286d6574616461...,Getting information about a table


In [544]:
table_info("AMMENDMENT_PROPOSALS")

,name,logic,description,expiry,votes
0,vote,0x0a64656620766f7465286d657461646174612c206375...,none,None,None
1,create_vote_collection_table,0x0a646566206372656174655f766f74655f636f6c6c65...,none,None,None


### Adding vote collection table

In [542]:
logic = '''
def create_vote_collection_table(metadata, cur, data):
    cur.execute("CREATE TABLE IF NOT EXISTS VOTES (user text, proposal_name text, vote int)")
    cur.execute(f'DELETE FROM RULES WHERE name="create_vote_collection_table"')
    return {"message":"hello world!"}
'''

name = "create_vote_collection_table"
create_ammendment_proposal(name, logic, 'none')

b'\x95\xf5\x1f\t\x9e\xb5\xe0\xcc\x99\xc0Wc\xb0\xc6v\xa2\xdaD\xec\xa4^\xe7\x85{\xcf\xbb7e`Vf\xdf'


In [545]:
ammendment_concensus(name)
time.sleep(10)
table_info("RULES")


b'\x0e\x15\xc0\x891m\xc6\r\x07=2tf\x11`@R\x08`&\x86\xfe\xbe\xf5\x92\xad}\x01N\x8c9\x98'


In [548]:
generate_call('input',"create_vote_collection_table" , {})
time.sleep(10)
table_info("RULES")

b'\xa9\x1f!\xb7\x8d\xa5\x0fh\xb6\xe3F\x8c\xaa\x10\x99L\x8f\xb5Z\xdb\xba\xf9T\xdc\xe8S\tW(\x9e\xd8\xe8'


In [549]:
time.sleep(10)
table_info("VOTES")

""


### Adding methods to vote

In [525]:
logic = '''
def vote(metadata, cur, data):
    voter = data['metadata']['msg_sender']
    proposal_name = metadata['proposal']
    vote = metadata['vote']
    
    # Make sure that voter is a valid voter
    verify = cur.execute('SELECT * FROM OWNERSHIPS WHERE depositor_address="{voter}"').fetchall()
    if len(verify)>0:
        if int(verify[0]["amount"])>5:
            cur.execute("INSERT INTO VOTES (user, proposal_name, vote) VALUES(?,?,?)",[voter, proposal_name, vote])
    else:
        return "Not allowed to vote"
        
'''

name = "vote"
create_ammendment_proposal(name, logic, 'none')

b'\xd7\xb4\xed\xcc>w\x9a\xf7\x7f\x1c\xda|\x9f\x1a\xed0Tt\xcaL\x17\xe3\xe3\xf0\x94rS\xb4\x913\xa9\xbd'


In [ ]:
ammendment_concensus(name)
generate_call('input',"create_vote_collection_table" , {})

### Updating concensus rule

In [ ]:
ammendment_concensus = '''
def ammendment_concensus(metadata, cur, data):
    proposal = metadata["name"]
    ammendment_data = cur.execute(f'SELECT (SUM(vote)/COUNT(vote)) AS vote_ratio FROM VOTES WHERE proposal_name="{proposal}"').fetchall()
    if len(ammendment_data>0):
        if ammendment_data[0]['vote_ratio']>0.5:
            ammendment_data = cur.execute(f'SELECT * FROM AMMENDMENT_PROPOSALS WHERE name="{proposal}"').fetchall()
            cur.execute("INSERT INTO RULES (name,logic,description) VALUES(?,?,?)",[ammendment_data[0]["name"], ammendment_data[0]["logic"], ammendment_data[0]["description"]])
            cur.execute(f'DELETE FROM AMMENDMENT_PROPOSALS WHERE name="{proposal}"')
            return f'Ammendment proposal {proposal} won'
        else:
            return f'Ammendment proposal {proposal} lost'
    else:
        return f'No votes for ammendment proposal {proposal}'
'''
name = "ammendment_concensus"
create_ammendment_proposal(name, logic, 'none')

In [ ]:
ammendment_concensus(name)

### Adding logic to make DAO self governing

# DAO L1 preparation Code

In [ ]:
import json
from web3 import Web3

# Connect to an Ethereum node or provider (e.g., Infura)
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))

### Accounts

In [ ]:
accounts = {
    "0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266":"0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80",
    "0x70997970C51812dc3A010C7d01b50e0d17dc79C8":"0x59c6995e998f97a5a0044966f0945389dc9e86dae88c7a8412f4603b6b78690d",
    "0x3C44CdDdB6a900fa2b585dd299e03d12FA4293BC":"0x5de4111afa1a4b94908f83103eb1f1706367c2e68ca870fc3fb9a804cdab365a",
    "0x90F79bf6EB2c4f870365E785982E1f101E93b906":"0x7c852118294e51e653712a81e05800f419141751be58f605c371e15141b007a6",
    "0x15d34AAf54267DB7D7c367839AAf71A00a2C6A65":"0x47e179ec197488593b187f80a00eb0da91f1b9d0b13f8733639f19c30a34926a",
    "0x9965507D1a55bcC2695C58ba16FB37d819B0A4dc":"0x8b3a350cf5c34c9194ca85829a2df0ec3153be0318b5e2d3348e872092edffba",
    "0x976EA74026E726554dB657fA54763abd0C3a0aa9":"0x92db14e403b83dfe3df233f83dfa3a0d7096f21ca9b0d6d6b8d88b2b4ec1564e",
    "0x14dC79964da2C08b23698B3D3cc7Ca32193d9955":"0x4bbbf85ce3377467afe5d46f804f221813b2bb87f24d81f60f1fcdbf7cbf4356",
    "0x23618e81E3f5cdF7f54C3d65f7FBc0aBf5B21E8f":"0xdbda1821b80551c9d65939329250298aa3472ba22feea921c0cf5d620ea67b97",
    "0xa0Ee7A142d267C1f36714E4a8F75612F20a79720":"0x2a871d0798f97d79848a013d4936a73bf4cc922c825d33c1cf7073dff6d409c6",
    "0xBcd4042DE499D14e55001CcbB24a551F3b954096":"0xf214f2b2cd398c806f84e317254e0f0b801d0643303237d97a22a48e01628897",
    "0x71bE63f3384f5fb98995898A86B02Fb2426c5788":"0x701b615bbdfb9de65240bc28bd21bbc0d996645a3dd57e7b12bc2bdf6f192c82",
    "0xFABB0ac9d68B0B445fB7357272Ff202C5651694a":"0xa267530f49f8280200edf313ee7af6b827f2a8bce2897751d06a843f644967b1",
    "0x1CBd3b2770909D4e10f157cABC84C7264073C9Ec":"0x47c99abed3324a2707c28affff1267e45918ec8c3f20b8aa892e8b065d2942dd",
    "0xdF3e18d64BC6A983f673Ab319CCaE4f1a57C7097":"0xc526ee95bf44d8fc405a158bb884d9d1238d99f0612e9f33d006bb0789009aaa",
    "0xcd3B766CCDd6AE721141F452C550Ca635964ce71":"0x8166f546bab6da521a8369cab06c5d2b9e46670292d85c875ee9ec20e84ffb61",
    "0x2546BcD3c84621e976D8185a91A922aE77ECEc30":"0xea6c44ac03bff858b476bba40716402b03e41b8e97e276d1baec7c37d42484a0",
    "0xbDA5747bFD65F08deb54cb465eB87D40e51B197E":"0x689af8efa8c651a91ad287602527f3af2fe9f6501a7ac4b061667b5a93e037fd",
    "0xdD2FD4581271e230360230F9337D5c0430Bf44C0":"0xde9be858da4a475276426320d5e9262ecfc3ba460bfac56360bfa6c4c28b4ee0",
    "0x8626f6940E2eb28930eFb4CeF49B2d1F2C9C1199":"0xdf57089febbacf7ba0bc227dafbffa9fc08a93fdc68e1e42411a14efcf23656e"
}

### Deploy this contract using Remix and get the deployment address

In [ ]:
# pragma solidity ^0.8.19;

# import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/master/contracts/token/ERC20/ERC20.sol";

# contract LiquidDAOToken is ERC20 {
#   constructor(string memory _name, string memory _symbol) ERC20(_name, _symbol) {
#     _mint(msg.sender, 1000 * 10 **18);
#   }

#     function mint_tokens(address account, uint256 value) public {
#         _mint(account, value);
#     }
  
# }

token_deployment_address = "0x851356ae760d987E095750cCeb3bC6014560891C"
token_abi = json.loads(open("/Users/shahnawazakhtar/Desktop/liquid-dao/abis/ERC20_Token.json").read())


### Mint tokens for all the addresses

In [ ]:
def mint_tokens(address_to_mint_for, token_deployment_address, token_abi, private_key):
    contract = w3.eth.contract(address=token_deployment_address, abi=token_abi)
    function = contract.functions['mint_tokens'](address_to_mint_for, 1000 * 10 **18)

    # Create a transaction
    transaction = function.buildTransaction({
        'chainId': 31337,  # Mainnet, change it to the desired network ID (e.g., 3 for Ropsten)
        'gas': 200000,  # Adjust the gas limit as required
        'gasPrice': w3.toWei('50', 'gwei'),  # Adjust the gas price as required
        'nonce': w3.eth.getTransactionCount(address_to_mint_for),
    })
    
    # Sign the transaction with your private key, sent it and wait for receipt
    signed_transaction = w3.eth.account.signTransaction(transaction, private_key=private_key)
    tx_hash = w3.eth.sendRawTransaction(signed_transaction.rawTransaction)
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    print(tx_hash)
    
for address, private_key in  accounts.items():
    mint_tokens(address, token_deployment_address, token_abi, private_key)

### Approve Cartesi Dapp to spend from all the accounts & then bridge to those accounts

In [ ]:
def approve_portal_to_spend(portal_address, token_deployment_address, token_abi, address, private_key):
    contract = w3.eth.contract(address=token_deployment_address, abi=token_abi)
    function = contract.functions['approve'](portal_address, 1000 * 10 **18)

    # Create a transaction
    transaction = function.buildTransaction({
        'chainId': 31337,  # Mainnet, change it to the desired network ID (e.g., 3 for Ropsten)
        'gas': 200000,  # Adjust the gas limit as required
        'gasPrice': w3.toWei('50', 'gwei'),  # Adjust the gas price as required
        'nonce': w3.eth.getTransactionCount(address),
    })
    
    # Sign the transaction with your private key, sent it and wait for receipt
    signed_transaction = w3.eth.account.signTransaction(transaction, private_key=private_key)
    tx_hash = w3.eth.sendRawTransaction(signed_transaction.rawTransaction)
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    print(tx_hash)
    
def bridge_tokens_to_cartesi(erc20_portal_address, erc20_portal_abi, cartesi_dapp_address, token_deployment_address, address, private_key):
    # parameters: IERC20 _token, address _dapp, uint256 _amount
    contract = w3.eth.contract(address=erc20_portal_address, abi=erc20_portal_abi)
    function = contract.functions['depositERC20Tokens'](token_deployment_address, cartesi_dapp_address, 1000000,'')

    # Create a transaction
    transaction = function.buildTransaction({
        'chainId': 31337,  # Mainnet, change it to the desired network ID (e.g., 3 for Ropsten)
        'gas': 200000,  # Adjust the gas limit as required
        'gasPrice': w3.toWei('50', 'gwei'),  # Adjust the gas price as required
        'nonce': w3.eth.getTransactionCount(address),
    })
    
    # Sign the transaction with your private key, sent it and wait for receipt
    signed_transaction = w3.eth.account.signTransaction(transaction, private_key=private_key)
    tx_hash = w3.eth.sendRawTransaction(signed_transaction.rawTransaction)
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    print(tx_hash)
    

In [ ]:
erc20_portal_abi = json.loads(open("/Users/shahnawazakhtar/Desktop/liquid-dao/deployments/localhost/ERC20Portal.json").read())["abi"]
erc20_portal_address = "0x4340ac4FcdFC5eF8d34930C96BBac2Af1301DF40"
dapp_address = "0x142105FC8dA71191b3a13C738Ba0cF4BC33325e2"    # From dapp.json file     


for address, private_key in  accounts.items():
    approve_portal_to_spend(erc20_portal_address, token_deployment_address, token_abi, address, private_key)
    time.sleep((5))
    bridge_tokens_to_cartesi(erc20_portal_address, erc20_portal_abi,dapp_address, token_deployment_address, address, private_key)

In [443]:
import requests
response = requests.post("http://localhost:5005/inspect/mypath", json={})


In [444]:
response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [445]:
const response = await fetch('http://localhost:4000/graphql', {
  method: 'POST',
  headers: { 'Content-Type': 'application/json' },
  body: '{ "query": "{ notices { edges { node { payload } } } }" }'
});
const result = await response.json();
for (let edge of result.data.notices.edges) {
    let payload = edge.node.payload;
}

<Response [405]>

In [459]:
response.status_code

400

In [461]:
response = requests.post("http://localhost:4000/graphql",json='{ "query": "{ notices { edges { node { payload } } } }" }')
response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [465]:
def t(name):
    data = {
        "action": "table_info",
        "metadata": {
            "name": name
        }}
    response = requests.post("http://localhost:5005/inspect", json=str2hex(json.dumps(data)))
    print(response.status_code)
    return response.json()
#     os.chdir("/Users/shahnawazakhtar/Desktop/liquid-dao/frontend-console")
#     os.system(f"yarn start inspect --payload '{json.dumps(data)}'")
t("RULES")

404


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [503]:
data = {
    "action": "table_info",
    "metadata": {
        "name": "RULES"
}}

response.status_code

200

,name,logic,description
0,create_ammendment_proposal,0x0a646566206372656174655f616d6d656e646d656e74...,Create ammendment proposal
1,delete_ammendment_proposal,0x0a6465662064656c6574655f616d6d656e646d656e74...,Delete ammendment proposal
2,ammendment_concensus,0x0a64656620616d6d656e646d656e745f636f6e63656e...,Rules for ammendment_concensus in the DAO at g...
3,table_info,0x0a646566207461626c655f696e666f286d6574616461...,Getting information about a table


In [471]:
data

'{"action": "create_ammendment_proposal", "metadata": {"name": "get_table", "logic": "\\n    def get_table(metadata, cur):\\n        table_name = metadata[\\"name\\"]\\n        cur.execute(\\"SELECT * FROM {}\\".format(table_name))\\n        return cur.fetchall()\\n    ", "description": "returns content of the table"}}'